In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
def get_rmse(y, y_pred, ddof=2):
    return np.sqrt( np.sum(np.square(y - y_pred)) / (len(y)-ddof) )


def model_filter(A, y, limrms=3, iters=2):
    '''
    limrms is 'how many RMSEs should be used to remove outliers'
    '''
    model = np.linalg.lstsq(A,y, rcond=False)[0]
    ddof = A.shape[1]
    y_pred = np.sum(A*model,axis=1)
    rmse = get_rmse(y, y_pred, ddof=ddof)
    for i in range(iters):
        count = len(y)
        # reducing dataset
        sel = np.abs(y_pred - y)<limrms*rmse
        y = y[sel]
        if len(y)==count:
            break
        A = A[sel]
        # second iteration (only)
        model = np.linalg.lstsq(A,y, rcond=False)[0]
        y_pred = np.sum(A*model,axis=1)
        rmse = get_rmse(y, y_pred, ddof=ddof)
    stderr = np.sqrt(rmse**2/len(y))
    return model, stderr


def get_s1b_offset(epd, fpd, col = 'daz_mm_notide_noiono', fix_pod_offset = True, rmsiter = 2,
                   split_by_pod = True, fit_offset = False, return_model = False, startfromnoiono = True, mincount = 80 ):
    '''
    epd = selected esd pandas dataframe
    fpd - selected frame pd df
    '''
    if fit_offset and fix_pod_offset:
        print('you do not want to do both..')
        return False
    try:
        epd = epd.set_index(epd.epochdate)
        epd = epd.sort_index()
    except:
        print('')
    if startfromnoiono:
        stdate=pd.Timestamp('2016-07-30').date()
        epd = epd[epd.index>stdate]
        if epd.empty:
            return np.nan
    dazes = epd[col].copy() #.values
    poddate = pd.Timestamp('2020-07-30')
    if fix_pod_offset:
        dazes[dazes.index<poddate]-=39
    epochdates = epd.index.values
    years = epd.years_since_beginning.values
    dazes = dazes.values
    masterdate = pd.Timestamp(fpd.master.values[0])
    mastersat = fpd.s1AorB.values[0]
    isB = flag_s1b(epochdates, masterdate, mastersat)
    if not split_by_pod:
        if fit_offset:
            is_pre = (epochdates<poddate).astype(np.int0)
            A = np.vstack((years,np.ones_like(years),isB, is_pre)).T
            model, stderr = model_filter(A, dazes, iters=rmsiter)
            #model = np.linalg.lstsq(A,dazes, rcond=False)[0]
            cAB = model[2]
            pod_offset = model[3]
            if not return_model:
                return pod_offset
        else:
            # now, we do d = A m, where A is of dt, 1, isB:
            A = np.vstack((years,np.ones_like(years),isB)).T
            model, stderr = model_filter(A, dazes, iters=rmsiter)
            #model = np.linalg.lstsq(A,dazes, rcond=False)[0]
    else:
        # now, we do d = A m, where A is of dt, 1, isB_pre, isB_post:
        isB_pre = (epochdates<poddate).astype(np.int0)*isB
        isB_post = (epochdates>=poddate).astype(np.int0)*isB
        minepochs = 10
        if (np.sum(isB_pre) < minepochs) or (np.sum(isB_post) < minepochs):
            return np.nan
        A = np.vstack((years,np.ones_like(years),isB_pre, isB_post)).T
        model, stderr = model_filter(A, dazes, iters=rmsiter)
        #model = np.linalg.lstsq(A,dazes, rcond=False)[0]
        cAB_pre = model[2]
        cAB_post = model[3]
        cdiff = cAB_post - cAB_pre
        if not return_model:
            return cdiff
    if not return_model:
        #v = model[0]
        #c = model[1]
        c_AB = model[2]
        if c_AB == 0:
            return np.nan
        else:
            return c_AB
    else:
        #get rmse -> stderr, return it
        return model, stderr




def flag_s1b(epochdates, masterdate, mastersat = 'A'):
    if mastersat == 'B':
        masterdate = masterdate + pd.Timedelta('6 days')
    isB = []
    for epoch in epochdates:
        # ok, give +- 1 day tolerance due to midnight issue
        if np.abs(np.mod((epoch - masterdate.date()).days, 12)) <= 1:
            isB.append(0)
        else:
            isB.append(1)
    isB = np.array(isB)
    return isB


In [ ]:
poddate = pd.Timestamp('2020-07-30')
esds = pd.read_csv('esds_202205/esds.csv')
#framess='/gws/nopw/j04/nceo_geohazards_vol1/projects/LiCS/proc/current/esds_2022/backup/s1aorb.csv'
#framess=pd.read_csv(framess)

esdir = '/gws/nopw/j04/nceo_geohazards_vol1/projects/LiCS/proc/current/esds_and_ranges_2020data'
inframesfile = os.path.join(esdir,'esds2021_frames_take4_withrg.csv')

framess = pd.read_csv(inframesfile)
#framespd 

mdates=[]
for frame in framess['frame']:
    mdate = fc.get_master(frame, asdate=True)
    mdates.append(mdate)

framess['master']=mdates
framess['opass'] = framess['frame'].str[3]
framespd = framess

In [ ]:
# to extend the dataset:
polyid = lq.sqlout2list(lq.get_frame_polyid(frame))
dazes = []
for epoch in b.epoch:
    dbdaz = lq.get_daz(polyid[0], epoch)
    dazes.append(dbdaz)

# or in full:
    
minBs = 25
minall = 75
poddate = pd.Timestamp('2020-07-30')
offsets = []
podoffs = []
counts = []
frames = []
for i,r in framess.iterrows():
    frame=r['frame']
    mastersat=r['s1AorB']
    masterdate=r['master']
    if not masterdate:
        continue
    # get the dazes:
    try:
        polyid = lq.sqlout2list(lq.get_frame_polyid(frame))[0]
        daztb = fc.lq.do_pd_query('select * from esd where polyid={};'.format(polyid))
        daztb = daztb.set_index(daztb.epoch).sort_index()
        dazes = (daztb['daz'])*14000
    except:
        continue
    if dazes.empty:
        continue
    stdate=pd.Timestamp('2016-07-30').date()
    dazes = dazes[dazes.index>stdate]
    if dazes.count()<minall:
        continue
    epochdates = dazes.index.values
    tdif = epochdates - epochdates[0]
    years = pd.DataFrame(tdif)[0].apply(lambda x: float(x.days)/365.25).values
    isB = flag_s1b(epochdates, masterdate, mastersat)
    if np.sum(isB)< minBs:
        continue
    is_pre = (epochdates<=poddate.date()).astype(np.int0)
    A = np.vstack((years,np.ones_like(years),isB, is_pre)).T
    model = np.linalg.lstsq(A,dazes, rcond=False)[0]
    cAB = model[2]
    podoff = model[3]
    offsets.append(cAB)
    counts.append(len(epochdates))
    podoffs.append(podoff)
    frames.append(frame)

In [ ]:
vels2 = []
offs2 = []
podoffs = []
stderrs = []
for frame in framespd['frame']:
    #print(frame)
    fpd = framespd[framespd['frame'] == frame]
    epd = esds[esds['frame'] == frame]
    #offsetdiff = get_s1b_offset(epd, fpd)
    modell, stderr = get_s1b_offset(epd, fpd, split_by_pod = False, fit_offset = True, fix_pod_offset = False, return_model = True)
    if type(modell) == type(np.nan):
        modell = [np.nan, np.nan, np.nan, np.nan]
        stderr = np.nan
    vels2.append(modell[0])
    offs2.append(modell[2])
    podoffs.append(modell[3])
    stderrs.append(stderr)

#print(offsets)
#framespd['s1ab_offset_mm_prepost'] = offsetdiffs
framespd['s1ab_offset_mm5'] = offs2
framespd['vel5'] = vels2
framespd['podoff5'] = podoffs
framespd['stderr5'] = stderrs


In [ ]:
framespd = framespd_selection.copy(deep=True)
#framespd_selection = framespd.copy(deep=True)
framespd = framespd[np.abs(framespd.vel5)<80]
framespd = framespd[np.abs(framespd.podoff5)<110]
framespd = framespd[np.abs(framespd.stderr5)<7]

#framespd = framespd[framespd.podoff5<75]
framespd=framespd[framespd.s1ab_offset_mm5>-50]

In [ ]:
dist=framespd[framespd['opass']=='A'].podoff5.rename('ascending frames')*-1
dist2=framespd[framespd['opass']=='D'].podoff5.rename('descending frames')*-1
dist3=framespd.podoff5.rename('all frames')*-1

plt.style.use('seaborn-white')
from palettable.colorbrewer.qualitative import Set2_7
colors = Set2_7.mpl_colors

params = {
   'axes.labelsize': 8,
   'font.size': 9,
   'legend.fontsize': 8,
   'figure.dpi' : 200,
   #'xtick.labelsize': 10,
   #'ytick.labelsize': 10,
   #'text.usetex': False,
   'figure.figsize': [3, 4]
   # 'figure.figsize': [6, 4]
   }
plt.rcParams.update(params)


fig, ax = plt.subplots()
#fig = plt.figure(figsize=(8,4), dpi=100)
#ax.set_xlim(-100,100)
bins1 = 12
bins = bins1*2
ax = dist.plot.hist(density=False, legend=True, color=colors[2], histtype='stepfilled',
                ax=ax, bins=bins1, grid=True, alpha=0.5, edgecolor='k', 
                label='ascending frames')
ax = dist2.plot.hist(density=False, legend=True, color=colors[4], histtype='stepfilled',
                ax=ax, bins=bins1, grid=True, alpha=0.5, edgecolor='k',
                label='descending frames')
#dist3.plot.hist(density=True, legend=True,
#                ax=ax, bins=60, grid=True, edgecolor='k', alpha=0.05, label='bagr')
ax = dist3.plot.hist(density=False, legend=True, color=colors[4], histtype='step',
                ax=ax, bins=bins, grid=True, alpha=0.85, edgecolor='red',linewidth=2,
                label='all frames')

               #title='Density histogram of $\Delta(\sigma_{post}-\sigma_{pre})$', 

median_both = dist3.median()
mean_both = dist3.mean()
print('median values are:')
print(dist.median())
print(dist2.median())
print(dist3.median())

print('mean values are:')
print(dist.mean())
print(dist2.mean())
print(dist3.mean())

print('count:')
print(dist3.count())



ax.set_xlim(-120,80)
ax.set_xlim(-100,100)
#ax.set_ylim(0,0.4)
#ax.set_title("Estimates of $\Delta a$ offset due to POD change")
ax.set_title("$\Delta a$ offset due to orbits change")
ax.grid(True)

stderr_median=np.sqrt(dist3.var()/dist3.count())*2*1.253
ax.axvline(median_both, color='k', linewidth=1, alpha = 0.7, linestyle='dashed', label='$\overline{\delta \Delta a}$ ='+str(int(np.round(median_both)))+'$\pm$'+str(np.round(stderr_median,1))+' mm')
#ax.axvline(mean_both, color='k', linewidth=1, alpha = 0.7, linestyle='dashed')
ax.set_ylim(0,90)
min_ylim, max_ylim = ax.get_ylim()
#ax.text(median_both+2, max_ylim*0.939, '$\overline{\delta \Delta a}$',
#        fontsize=11)
#ax.text(median_both-14, max_ylim*0.939, '$\overline{\delta \Delta a}$',
#        fontsize=11)
#ax.axvline(mean_both, color='k', linewidth=1, alpha = 0.7, linestyle='dashed')
#ax.text(mean_both, max_ylim*0.95, ' $\mu_{\Delta,post}=$'+'{:.2f} mm'.format(mean_both),
#        fontsize=9)

#the daz_ARP = -39 mm
ax.axvline(-39, color='b', linewidth=1, alpha = 0.7, label='$\Delta a_{ARP}$ = -39 mm')
#ax.text(-39+2, max_ylim*0.945, '$daz_{ARP}$ = -39 mm', color='b', fontsize=11)
#ax.text(-39+1, max_ylim*0.941, '$\Delta a_{ARP}$', color='b', fontsize=11)
#ax.legend(facecolor='white', framealpha=1, loc='lower left')
legend = ax.legend(frameon = 1, loc='upper right')
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_linewidth(0)
ax.set_xlabel('mm')